In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import hvplot.pandas
import scipy.stats as st
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import linregress
from scipy import stats
import holoviews as hv

In [ ]:
# Take in all of our world energy data and read it into pandas

world_energy_df = pd.read_csv("Resources/world_energy_consumption.csv")
world_energy_df

In [ ]:
world_energy_df.columns.tolist()

In [ ]:
pd.set_option("display.max_columns", None)
world_energy_df.info(verbose=True)

In [ ]:
#See how our data looks for unavailable cells
combined_conditions = world_energy_df[
                    world_energy_df.isna().any(axis=1) | (world_energy_df == '').any(axis=1) | (world_energy_df == 0).any(axis=1)]
combined_conditions

In [ ]:
world_energy_df=world_energy_df.fillna(0)

In [ ]:
world_energy_df['non_renewables_consumption'] = (
    world_energy_df['coal_consumption'] + world_energy_df['gas_consumption']+ world_energy_df['oil_consumption'] + world_energy_df['nuclear_consumption']
)
world_energy_df['Total Energy Consumption'] = (
    world_energy_df['renewables_consumption'] + world_energy_df['non_renewables_consumption'] 
)


In [ ]:
world_energy_df['non_renewables_production'] = (
    world_energy_df['coal_electricity'] + world_energy_df['coal_production'] + world_energy_df['gas_production'] +
    world_energy_df['gas_electricity'] + world_energy_df['nuclear_electricity'] + world_energy_df['oil_electricity'] + world_energy_df['oil_production']
)

world_energy_df['Total Energy Production'] = (
    world_energy_df['renewables_electricity'] + world_energy_df['non_renewables_production'] 
)

world_energy_df

In [ ]:
# Create a new column for the non-renewable energy production, total_ per capita
world_energy_df['non-renewables_prod_per_capita'] = (
    world_energy_df['oil_prod_per_capita'] + world_energy_df['coal_elec_per_capita'] + world_energy_df['gas_elec_per_capita'] + world_energy_df['nuclear_elec_per_capita']
)
world_energy_df['total_prod_per_capita'] = (
    world_energy_df['non-renewables_prod_per_capita'] + world_energy_df['renewables_elec_per_capita']
)
world_energy_df['non-renewables_cons_per_capita'] = (
    world_energy_df['oil_energy_per_capita'] + world_energy_df['coal_cons_per_capita'] + world_energy_df['gas_energy_per_capita'] + world_energy_df['nuclear_energy_per_capita']
)
world_energy_df['total_cons_per_capita'] = (
    world_energy_df['non-renewables_cons_per_capita'] + world_energy_df['renewables_energy_per_capita'])

world_energy_df.head()

In [ ]:
pd.set_option("display.max_columns", None)
count_result = world_energy_df.count()
count_result

In [ ]:
countries_world_energy_df = world_energy_df.loc[(world_energy_df['iso_code'] != 0) ]
countries_world_energy_df


In [ ]:
countries_world_energy_df=countries_world_energy_df.loc[(countries_world_energy_df["year"] >= 1985)] 
countries_world_energy_df.head()

In [ ]:
total_world_consumption=countries_world_energy_df['Total Energy Consumption'].sum()
total_world_production=countries_world_energy_df['Total Energy Production'].sum()
total_world_renewable_production=countries_world_energy_df['renewables_electricity'].sum()
total_world_renewable_consumption=countries_world_energy_df['renewables_consumption'].sum()
print(f'Total World Energy Consumption is: {round(total_world_consumption,0)}')
print(f'Total World Energy Production is: {round(total_world_production,0)}')
print(f'Total World Renewable-Energy Production is: {round(total_world_renewable_production,0)}')
print(f'Total World Renewable-Energy Consumption is: {round(total_world_renewable_consumption,0)}')

In [ ]:
top_country_df = (
    countries_world_energy_df
    .groupby('country', as_index=False)['Total Energy Production']
    .sum()
    .sort_values(by='Total Energy Production', ascending=False)
)
top_country_df.head(25)

In [ ]:
top10_country_production=countries_world_energy_df.loc[
    countries_world_energy_df['country'].isin([
        'China', 'United States', 'Russia', 'India', 'Saudi Arabia', 'Canada', 'Australia', 'Iran', 'Indonesia', 'Mexico'
    ])]
top10_country_production

In [ ]:
top_country_df = (
    countries_world_energy_df
    .groupby('country', as_index=False)['Total Energy Consumption']
    .sum()
    .sort_values(by='Total Energy Consumption', ascending=False)
)
top_country_df.head(25)

In [ ]:
top10_country_consumption=countries_world_energy_df.loc[
    countries_world_energy_df['country'].isin([
        'China', 'United States', 'Russia', 'India', 'Japan', 'Canada', 'Germany', 'France', 'United Kingdom', 'Brazil'
    ])]
top10_country_consumption

In [ ]:
top_country_df = countries_world_energy_df.loc[
    countries_world_energy_df['country'].isin([
        'China', 'United States', 'Russia', 'India', 'Saudi Arabia', 'Canada', 'Australia', 'Iran', 'Indonesia', 'Mexico',
        'United Kingdom', 'Norway', 'Venezuela', 'United Arab Emirates', 'South Africa', 'Germany', 'Algeria', 'Iraq', 'Brazil', 'Kuwait',
        'Japan', 'France', 'South Korea', 'Italy', 'Ukraine', 'Spain', 'Poland', 'South Africa', 'Netherlands', 'Turkey', 'Taiwan', 
        'Thailand', 'Nigeria', 'Kazakhstan', 'Qatar'
    ])
]


top_countries_total_consumption=top_country_df['Total Energy Consumption'].sum()
top_countries_total_production=top_country_df['Total Energy Production'].sum()
print(f'Top Countries Total Consumption Overall is: {round((top_countries_total_consumption / total_world_consumption) * 100)}%')
print(f'Top Countries Total Production Overall is: {round((top_countries_total_production / total_world_production) * 100)}%')
                                                              
                                      

In [ ]:
#Find the top countries that are sorted by renewable energy consumption
top_countries_renewable = (
    countries_world_energy_df
    .loc[countries_world_energy_df['year'].between(2008, 2022)]
    .groupby('country', as_index=False)['renewables_electricity']
    .sum()
    .sort_values(by='renewables_electricity', ascending=False)
)

top_countries_renewable.head(15)

In [ ]:
top_countries_renewable = countries_world_energy_df.loc[
    countries_world_energy_df['country'].isin([
        'China', 'United States', 'Brazil', 'Canada', 'Russia', 'India', 'Norway', 'Japan', 'Germany', 
        'Sweden', 'France', 'Italy', 'Spain', 'United Kingdom', 'Turkey'
    ])
]
top_countries_renewable

In [ ]:
top_countries_renewable_cons=top_countries_renewable['renewables_consumption'].sum()
top_countries_renewable_prod=top_countries_renewable['renewables_electricity'].sum()
print(f'Renewable Energy Over Total World Renewable Energy Consumption: {round(top_countries_renewable_cons / total_world_renewable_consumption * 100)}%')
print(f'Renewable Energy Over Total World Renewable Energy Production: {round(top_countries_renewable_prod / total_world_renewable_production * 100)}%')


In [ ]:
#VISUAL (EYLEM)
#Create a pie graph for renewable and non-renewable energy consumption for 2018-2022 years

filt_countries_world_energy=countries_world_energy_df[countries_world_energy_df['year'] >= 2018]
pie20182022=filt_countries_world_energy[['renewables_consumption', 'non_renewables_consumption']].sum()

# Pie chart configuration
explode = (0, 0)  
colors = ["yellow", "lightcoral"]

# Plot the pie chart
plt.figure(figsize=(8, 6))

plt.pie(pie20182022, labels=pie20182022.index, 
        colors=colors, autopct="%1.1f%%", startangle=90)

# Ensure the pie chart is circular
plt.axis("equal")  

#Save the plot
plt.savefig('Resources/e_pie_both.png')

# Show the plot
plt.show()

In [ ]:
#VISUAL (EYLEM)
#Create a pie graph for top countries for renewable energy consumption
renewableenergy=filt_countries_world_energy[['biofuel_consumption', 'hydro_consumption', 
                                                     'solar_consumption', 'wind_consumption', 'other_renewable_consumption']].sum()

# Pie chart configuration
explode = (0.05, 0, 0, 0, 0)  # "explode" the first slice (biofuel)
colors = ["yellow", "lightcoral", "green", "purple", "orange"]

# Plot the pie chart
plt.figure(figsize=(8, 6))

plt.pie(renewableenergy, explode=explode, labels=renewableenergy.index, 
        colors=colors, autopct="%1.1f%%", startangle=90)

# Ensure the pie chart is circular
plt.axis("equal")  

#Save the plot
plt.savefig('Resources/renewable_energy_top_country.png')

# Show the plot
plt.show()

In [ ]:
#VISUAL (EYLEM)
#Create a pie graph for top countries for renewable energy production categories
nonrenewableenergy=top_countries_renewable[['coal_production', 'gas_production', 
                                                     'oil_production', 'nuclear_electricity']].sum()
# Pie chart configuration
explode = (0.05, 0, 0, 0)  # "explode" the first slice
colors = ["yellow", "lightcoral", "green", "purple"]

# Plot the pie chart
plt.figure(figsize=(8, 6))
plt.title("Non-Renewable Energy Production in Top Countries")
plt.pie(nonrenewableenergy, explode=explode, labels=nonrenewableenergy.index, 
        colors=colors, autopct="%1.1f%%", startangle=90)

#Save the plot as png
plt.savefig('Resources/e_topcountryrnw.png')
# Ensure the pie chart is circular
plt.axis("equal")  

#Save the plot
plt.savefig('Resources/non-renewable-production.png')

# Show the plot
plt.show()

In [ ]:
#VISUAL (EYLEM)
#Create a bar graph to show hydro energy production and consumption values by countries


# Group by country and sum hydpro energy values
hydro_bar = top_countries_renewable.groupby('country')[['hydro_electricity', 'hydro_consumption']].sum()

# Plot
hydro_bar.plot(kind='bar', figsize=(8, 6))
plt.title('Top Countries by Hydro Production and Consumption')
plt.xlabel('Year')
plt.ylabel('Units')

#Save the plot
plt.savefig('Resources/top_country_hydpro.png')

plt.show()

In [ ]:
#VISUAL (EYLEM)
# Plot energy production over time by Country

production_plot_country = top10_country_production.hvplot.line(
    x='year', y='Total Energy Consumption', by='country',
    title="Total Energy Consumption Over Time by Country",
    xlabel='Year', ylabel='Energy Consumption',
    line_width=2, height=400, width=800
)
plt.savefig('Resources/total_energy_cons_country.png')
production_plot_country

In [ ]:
#Group by 'year' and calculate the sum for each year
cons_prod_df = top_country_df.groupby('year')[['Total Energy Production', 'Total Energy Consumption','renewables_electricity',
                                               'non_renewables_production','renewables_consumption','non_renewables_consumption' ]].sum()
cons_prod_df = cons_prod_df.reset_index()
cons_prod_df['non_renewables_consumption'] = pd.to_numeric(cons_prod_df['non_renewables_consumption'], errors='coerce')
cons_prod_df['non_renewables_production'] = pd.to_numeric(cons_prod_df['non_renewables_production'], errors='coerce')
cons_prod_df

In [ ]:
# VISUAL (Amy)
top_country_filtered=top_country_df.loc[top_country_df['year'] < 2019 ]

# Visuals (Amy)
top_country_filtered=top_country_df.loc[top_country_df['year'] < 2019 ]
# Filter data for the specific year (1995)
year_to_plot = 1995
year_filtered_df = top_country_filtered[top_country_filtered['year'] == year_to_plot]
# Select relevant columns for consumption and production
filtered_df = year_filtered_df[['country', 'year', 'total_cons_per_capita', 'total_prod_per_capita']]
# Set the bar width and space between
bar_width = 0.4
gap = 1.0
# Set positions of bars on the X-axis
r1 = [x * gap for x in range(len(filtered_df))]  # Positions for consumption
r2 = [x + bar_width + gap for x in r1]  # Positions for production (shifted by bar_width)
# Increase figure size to allow more space for labels
plt.figure(figsize=(11, 7))
# Create the bar chart
plt.bar(r1, filtered_df['total_cons_per_capita'], color='teal', width=bar_width, edgecolor='grey', label='Consumption')
plt.bar(r2, filtered_df['total_prod_per_capita'], color='darkorange', width=bar_width, edgecolor='grey', label='Production')
# Add labels and title
plt.xlabel('Countries', fontweight='bold')
plt.xticks([r + bar_width for r in range(len(filtered_df))], filtered_df['country'], rotation = 90, ha = 'center')
plt.ylabel('Energy (per capita)', fontweight='bold')
plt.title(f'Energy Consumption and Production per Capita in {year_to_plot}')
# Adjust layout to make room for longer labels
plt.tight_layout()
plt.legend()
# Show the plot
plt.show()
#plt.clf()
# Filter data for the specific year (2005)
year_to_plot = 2005
year_filtered_df = top_country_df[top_country_df['year'] == year_to_plot]
# Select relevant columns for consumption and production
filtered_df = year_filtered_df[['country', 'year', 'total_cons_per_capita', 'total_prod_per_capita']]
# Set the bar width and space between
bar_width = 0.4
gap = 1.0
# Set positions of bars on the X-axis
r1 = [x * gap for x in range(len(filtered_df))]  # Positions for consumption
r2 = [x + bar_width + gap for x in r1]  # Positions for production (shifted by bar_width)
# Increase figure size to allow more space for labels
plt.figure(figsize=(12, 7))
# Create the bar chart
plt.bar(r1, filtered_df['total_cons_per_capita'], color='teal', width=bar_width, edgecolor='grey', label='Consumption')
plt.bar(r2, filtered_df['total_prod_per_capita'], color='darkorange', width=bar_width, edgecolor='grey', label='Production')
# Add labels and title
plt.xlabel('Countries', fontweight='bold')
plt.xticks([r + bar_width for r in range(len(filtered_df))], filtered_df['country'], rotation = 90, ha = 'center')
plt.ylabel('Energy (per capita)', fontweight='bold')
plt.title(f'Energy Consumption and Production per Capita in {year_to_plot}')
# Adjust layout to make room for longer labels
plt.tight_layout()
# Adjust the y-axis limit to match the 1995 y axis for better comparison
plt.ylim(0, 700000 * 1.1)
plt.legend()
# Show the plot
plt.show()
#plt.clf()
# Filter data for the specific year (2018)
year_to_plot = 2018
year_filtered_df = top_country_df[top_country_df['year'] == year_to_plot]
# Select relevant columns for consumption and production
filtered_df = year_filtered_df[['country', 'year', 'total_cons_per_capita', 'total_prod_per_capita']]
# Set the bar width and space between
bar_width = 0.4
gap = 1.0
# Set positions of bars on the X-axis
r1 = [x * gap for x in range(len(filtered_df))]  # Positions for consumption
r2 = [x + bar_width + gap for x in r1]  # Positions for production (shifted by bar_width)
# Increase figure size to allow more space for labels
plt.figure(figsize=(12, 7))
# Create the bar chart
plt.bar(r1, filtered_df['total_cons_per_capita'], color='teal', width=bar_width, edgecolor='grey', label='Consumption')
plt.bar(r2, filtered_df['total_prod_per_capita'], color='darkorange', width=bar_width, edgecolor='grey', label='Production')
# Add labels and title
plt.xlabel('Countries', fontweight='bold')
plt.xticks([r + bar_width for r in range(len(filtered_df))], filtered_df['country'], rotation = 90, ha = 'center')
plt.ylabel('Energy (per capita)', fontweight='bold')
plt.title(f'Energy Consumption and Production per Capita in {year_to_plot}')
# Adjust layout to make room for longer labels
plt.tight_layout()
# Adjust the y-axis limit to match the 1995 y axis for better comparison
plt.ylim(0, 700000 * 1.1)
plt.legend()
# Show the plot
plt.show()


In [ ]:
# Q1 (Amy) What is the relationship between energy production per capita and energy consumption per capita across countries?
# Statistical Analysis of per capita consumption and production

#Dataframe for per capita consumption and production for top countries over the years
filtered_df = top_country_filtered[['country', 'year', 'total_cons_per_capita', 'total_prod_per_capita']]

# Descriptive statistics for both consumption and production per capita
consumption_stats = filtered_df['total_cons_per_capita'].describe()
production_stats = filtered_df['total_prod_per_capita'].describe()



In [ ]:
# Q1 (Amy)
# Plotting the scatter plot
plt.figure(figsize=(7, 10))
# Scatter plot of % change in GDP vs % change in renewable energy consumption
plt.scatter(top_country_filtered['total_cons_per_capita'], top_country_filtered['total_prod_per_capita'], c='teal', s=100, label='Data points')
# Add labels and title
plt.title('Correlation between Per Capita Energy Consumption and Production')
plt.xlabel('Energy Production (kWh)')
plt.ylabel('Energy Consumption (kWh)')
# Add a linear regression line (Fit a line to the scatter plot)
try:
    slope, intercept = np.polyfit(top_country_filtered['total_cons_per_capita'], top_country_filtered['total_prod_per_capita'], 1)
    plt.plot(top_country_filtered['total_cons_per_capita'], slope * top_country_filtered['total_cons_per_capita'] + intercept, color='red', label='Regression Line')
except Exception as e:
    print(f"Error fitting regression line: {e}")
# Show legend
plt.legend()
# Show the plot
plt.tight_layout()
plt.show()
# Calculate the Pearson correlation coefficient between energy production and consumption per capita
correlation = top_country_filtered[['total_prod_per_capita', 'total_cons_per_capita']].corr().iloc[0, 1]
print(f"Correlation between per capita energy production and per capita energy consumption: {correlation:.2f}")

In [ ]:
#Q3 (Amy) Is there a correlation between renewable energy consumption and GDP growth?
# Plotting the scatter plot
plt.figure(figsize=(7, 10))
# Scatter plot of % change in GDP vs % change in renewable energy consumption
plt.scatter(top_country_filtered['gdp'], top_country_filtered['renewables_consumption'], c='darkgreen', s=100, label='Data points')
# Add labels and title
plt.title('Correlation between GDP and Renewable Energy Consumption')
plt.xlabel('GDP')
plt.ylabel('Renewable Energy Consumption')
# Add a linear regression line (Fit a line to the scatter plot)
try:
    slope, intercept = np.polyfit(top_country_filtered['gdp'], top_country_filtered['renewables_consumption'], 1)
    plt.plot(top_country_filtered['gdp'], slope * top_country_filtered['gdp'] + intercept, color='red', label='Regression Line')
except Exception as e:
    print(f"Error fitting regression line: {e}")
# Show legend
plt.legend()
# Show the plot
plt.tight_layout()
plt.show()
# Calculate the Pearson correlation coefficient between % change in GDP and renewable energy consumption
correlation = top_country_filtered[['gdp', 'renewables_consumption']].corr().iloc[0, 1]
print(f"Correlation between GDP and renewable energy consumption: {correlation:.2f}")

In [ ]:
#Q3 (Amy)
# Step 1: Calculate the average GDP and average renewable energy consumption for all countries combined, by year
avg_gdp = top_country_filtered.groupby('year')['gdp'].mean().reset_index()
avg_renewables_consumption = top_country_filtered.groupby('year')['renewables_consumption'].mean().reset_index()
# Step 2: Create a subplot with two axes
fig, ax1 = plt.subplots(figsize=(10, 6))
# Plotting the average GDP on the left y-axis
ax1.plot(avg_gdp['year'], avg_gdp['gdp'], label='Average GDP', linestyle='-', color='b')
ax1.set_xlabel('Year')
ax1.set_ylabel('Average GDP', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_title('Average GDP and Renewable Energy Consumption (1985-2018)')
# Create a second axis for renewable energy consumption
ax2 = ax1.twinx()  # Instantiate second axis
ax2.plot(avg_renewables_consumption['year'], avg_renewables_consumption['renewables_consumption'], label='Average Renewable Energy Consumption', linestyle='-', color='g')
ax2.set_ylabel('Average Renewable Energy Consumption (TWh)', color='g')
ax2.tick_params(axis='y', labelcolor='g')
# Add legends for both axes
ax1.legend(loc='upper left')
ax2.legend(loc='lower right')
# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
#Q3 (Amy) Plotting top 6 countries by GDB and renewable energy comsumption
countries_to_plot = ['United States', 'China', 'Russia']
countries_to_plot_2 = ['Japan', 'India', 'Germany']
# Create a subplot with two axes
fig, ax1 = plt.subplots(figsize=(8, 5))
# Plotting GDP on the left y-axis
for country in countries_to_plot:
    country_data = top_country_filtered[top_country_filtered['country'] == country]
    ax1.plot(country_data['year'], country_data['gdp'], label=f'{country} - GDP', linestyle='-', marker='o')
ax1.set_xlabel('Year')
ax1.set_ylabel('GDP')
ax1.set_title('GDP and Renewable Energy Consumption by Country')
ax1.legend(loc='best')
# Create a second axis for renewable consumption
ax2 = ax1.twinx()  # Instantiate second axis
for country in countries_to_plot:
    country_data = top_country_filtered[top_country_filtered['country'] == country]
    ax2.plot(country_data['year'], country_data['renewables_consumption'], label=f'{country} - Renewables Cons', linestyle='-')
ax2.set_ylabel('Renewable Energy Consumption (TWh)')
ax2.legend(loc='upper right')
# Display the plot
plt.tight_layout()
plt.show()
# Create a subplot with two axes
fig, ax1 = plt.subplots(figsize=(8, 5))
# Plotting GDP on the left y-axis
for country in countries_to_plot_2:
    country_data = top_country_filtered[top_country_filtered['country'] == country]
    ax1.plot(country_data['year'], country_data['gdp'], label = f'{country} - GDP', linestyle='-', marker='o')
ax1.set_xlabel('Year')
ax1.set_ylabel('GDP')
ax1.set_title('GDP and Renewable Energy Consumption by Country')
ax1.legend(loc='best')
# Create a second axis for renewable consumption
ax2 = ax1.twinx()  # Instantiate second axis
for country in countries_to_plot_2:
    country_data = top_country_filtered[top_country_filtered['country'] == country]
    ax2.plot(country_data['year'], country_data['renewables_consumption'], label=f'{country} - Renewables Cons', linestyle='-')
ax2.set_ylabel('Renewable Energy Consumption (TWh)')
ax2.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
# Plot using hvplot showing consumption and production graph by renewable and non-renewable energy
consumption_production_plot = cons_prod_df.hvplot.line(
    x='year', 
    y=['renewables_electricity', 'non_renewables_production', 'renewables_consumption', 'non_renewables_consumption'],
    color=['blue', 'orange', 'green', 'red'],  # Colors for each line
    legend='top_left',  # Position of the legend
    line_width=2,
    xlabel='Year', 
    ylabel='Total Energy',
    title='Total Energy Production and Consumption Over Years',
    height=400, 
    width=800
)

# Configure x-ticks every 5 years
consumption_production_plot = consumption_production_plot.opts(xticks=dict(values=cons_prod_df['year'][::5]))
#Save the plot
plt.savefig('Resources/e_totalenergyline')
# Show the plot
consumption_production_plot

In [ ]:
#Q4-1 Plotting energy production and consumption values with the major events globally
plt.figure(figsize=(10, 6))
plt.plot(cons_prod_df['year'], cons_prod_df['renewables_electricity'], label='Total Renewable-Energy Production', color='blue')
plt.plot(cons_prod_df['year'], cons_prod_df['non_renewables_production'], label='Total Non-Renewable-Energy Production', color='orange')
plt.plot(cons_prod_df['year'], cons_prod_df['renewables_consumption'], label='Total Renewable-Energy Consumption', color='green')
plt.plot(cons_prod_df['year'], cons_prod_df['non_renewables_consumption'], label='Total Non-Renewable-Energy Consumption', color='red')


# Highlighting events
events = {2008: 'Global Financial Crises', 2014: 'Oil Price Collapse', 2019: 'COVID-19'}  
for year, event in events.items():
    plt.axvline(x=year, color='black', linestyle='--', alpha=0.7)
    plt.text(year, plt.ylim()[1]*0.8, event, rotation=45, color='black', ha='center')

# Labels and title
plt.xlabel('Year')
plt.ylabel('Energy (Units)')
plt.title('Energy Production and Consumption with Major Events')

plt.legend(loc='upper left')

#Save the plot as image file
plt.savefig('Resources/e_majorevents')

# Show the plot 
plt.tight_layout() 
plt.show()

In [ ]:
# Q4-2: Create events data and create the dataframe for our analysis
events = {
    2014: "Oil Price Collapse",
    2008: "Global Financial Crisis",
    2019: "COVID-19 Pandemic"
}
events_df = pd.DataFrame(list(events.items()), columns=['year', 'event'])

# Merge the dataframe with cons_prod_df
events_merge_df = cons_prod_df.merge(events_df[['year']], on='year', how='left')
events_merge_df['event_indicator'] = cons_prod_df['year'].isin(events_df['year']).astype(int)



# Create a new column for lagged event indicators
def create_lagged_event_indicator(events_merge_df, lag_years=2):
    """
    Creates lagged event indicators for up to `lag_years` after the event.
    For example, if an event occurred in 2014, it will flag the years 2015, 2016, etc.
    """
    for lag in range(1, lag_years + 1):
        events_merge_df[f'event_indicator_lag_{lag}'] = events_merge_df['event_indicator'].shift(+lag, fill_value=0)
    return events_merge_df

# Add lagged event indicators (for 1 and 2 years)
events_merge_df = create_lagged_event_indicator(events_merge_df, lag_years=2)

# Check the DataFrame
events_merge_df.head(10)


In [ ]:
#Q4-3 T-test with Lagged Years:
#Define event years and non-event years
event_years = events_merge_df[events_merge_df['event_indicator'] == 1]
non_event_years = events_merge_df[events_merge_df['event_indicator'] == 0]

#Define lagged years
lag_1_event_years = events_merge_df[events_merge_df['event_indicator_lag_1'] == 1]
lag_2_event_years = events_merge_df[events_merge_df['event_indicator_lag_2'] == 1]

# T-test for lag 1 years (Non-Renewable Energy Production)
t_stat_prod_lag_1, p_val_prod_lag_1 = st.ttest_ind(lag_1_event_years['non_renewables_production'], non_event_years['non_renewables_production'], equal_var=False)
print(f"T-test for Non-Renewable Energy Production (Lag 1) - T-statistic: {t_stat_prod_lag_1}, P-value: {p_val_prod_lag_1}")

# T-test for lag 2 years (Non-Renewable Energy Production)
t_stat_prod_lag_2, p_val_prod_lag_2 = st.ttest_ind(lag_2_event_years['non_renewables_production'], non_event_years['non_renewables_production'], equal_var=False)
print(f"T-test for Non-Renewable Energy Production (Lag 2) - T-statistic: {t_stat_prod_lag_2}, P-value: {p_val_prod_lag_2}")


# T-test for lag 1 years (Non-Renewable Energy Consumption)
t_stat_prod_lag_1, p_val_prod_lag_1 = st.ttest_ind(lag_1_event_years['non_renewables_consumption'], non_event_years['non_renewables_consumption'], equal_var=False)
print(f"T-test for Non-Renewable Energy Consumption (Lag 1) - T-statistic: {t_stat_prod_lag_1}, P-value: {p_val_prod_lag_1}")

# T-test for lag 2 years (Non-Renewable Energy Consumption)
t_stat_prod_lag_2, p_val_prod_lag_2 = st.ttest_ind(lag_2_event_years['non_renewables_consumption'], non_event_years['non_renewables_consumption'], equal_var=False)
print(f"T-test for Non-Renewable Energy Consumption (Lag 2) - T-statistic: {t_stat_prod_lag_2}, P-value: {p_val_prod_lag_2}")

In [ ]:
#Q4-3 Regression Analysis with Lagged Variables:
# Define the independent variable (lagged event indicators) and dependent variable (energy production or consumption)
X_prod = events_merge_df[['event_indicator', 'event_indicator_lag_1', 'event_indicator_lag_2']]
y_prod = events_merge_df['non_renewables_production']

X_cons = events_merge_df[['event_indicator', 'event_indicator_lag_1', 'event_indicator_lag_2']]
y_cons = events_merge_df['non_renewables_consumption']

# Add constant (intercept) to the independent variable
X_prod = sm.add_constant(X_prod)
X_cons = sm.add_constant(X_cons)

# Fit the regression model for production
model_prod = sm.OLS(y_prod, X_prod).fit()

# Fit the regression model for consumption
model_cons = sm.OLS(y_cons, X_cons).fit()

# Print the regression results
print("Regression results for Non-Renewable Energy Production:")
print(model_prod.summary())

print("Regression results for Non-Renewable Energy Consumption:")
print(model_cons.summary())

In [ ]:
#Q5 (Eylem) What are the projected renewable energy consumption trends for the next 50 years based on current data?

# Ensure 'year' and 'renewable_energy_consumption' are numeric
cons_prod_df['year'] = pd.to_numeric(cons_prod_df['year'], errors='coerce')
cons_prod_df['renewables_consumption'] = pd.to_numeric(cons_prod_df['renewables_consumption'], errors='coerce')

# Drop any rows with NaN values in the relevant columns
cons_prod_df = cons_prod_df.dropna(subset=['year', 'renewables_consumption'])

# Fit a linear regression model to the data
slope, intercept, r_value, p_value, std_err = linregress(cons_prod_df['year'], cons_prod_df['renewables_consumption'])

# Define the range of years for the next 50 years
future_years = np.arange(cons_prod_df['year'].max() + 1, cons_prod_df['year'].max() + 51)

# Calculate projected renewable energy consumption
projected_consumption = slope * future_years + intercept

# Plot the historical data and the projection
plt.figure(figsize=(12, 6))
plt.plot(cons_prod_df['year'], cons_prod_df['renewables_consumption'], label='Historical Data', color='blue')
plt.plot(future_years, projected_consumption, label='Projected Trend (Next 50 Years)', color='orange', linestyle='--')
plt.xlabel('Year')
plt.ylabel('Renewable Energy Consumption')
plt.title('Projected Renewable Energy Consumption Trends for the Next 50 Years')
plt.legend()
plt.grid(True)
#Save the plot
plt.savefig("Resources/e_projected_ren-energy_cons.png")

plt.show()


In [ ]:
#Q5 (Eylem)
# Create a DataFrame with the projected years and renewable energy consumption
projected_df = pd.DataFrame({
    'year': future_years,
    'projected_renewables_consumption': projected_consumption
})

# Export the DataFrame to a CSV file
projected_df.to_csv("Resources/projected_renewable_energy_consumption.csv", index=False)

# Display the first few rows of the DataFrame for verification
projected_df.tail()

In [ ]:
#Q5 (Eylem)
#Find the percentage change in year 2072 and 2022 (last reported data)
# Define the years for calculation

year_2072 = 2072

# Calculate projected consumption for 2072 and 2020 using the linear regression model
consumption_2022 = cons_prod_df.loc[cons_prod_df['year'] == 2022, 'renewables_consumption'].values
consumption_2072 = slope * year_2072 + intercept

# Calculate the percentage change
percentage_change = ((consumption_2072 - consumption_2022) / consumption_2022) * 100

# Display the result
print(f"Projected percentage change in renewable energy consumption in 2072: {round(float(percentage_change),2)}%")


In [ ]:
#Q5 (Eylem) find for non-renewables energy consumption

# Ensure 'year' and 'renewable_energy_consumption' are numeric

cons_prod_df['non_renewables_consumption'] = pd.to_numeric(cons_prod_df['non_renewables_consumption'], errors='coerce')

# Drop any rows with NaN values in the relevant columns
cons_prod_df = cons_prod_df.dropna(subset=['year', 'non_renewables_consumption'])

# Fit a linear regression model to the data
slope, intercept, r_value, p_value, std_err = linregress(cons_prod_df['year'], cons_prod_df['non_renewables_consumption'])

# Define the range of years for the next 50 years
future_years_nonrenw = np.arange(cons_prod_df['year'].max() + 1, cons_prod_df['year'].max() + 51)

# Calculate projected renewable energy consumption
projected_consumption_nonrenw = slope * future_years_nonrenw + intercept

# Create a DataFrame with the projected years and non-renewable energy consumption
projected_nonrenewable_df = pd.DataFrame({
    'year': future_years_nonrenw,
    'projected_consumption_nonrenw': projected_consumption_nonrenw
})


# Display the first few rows of the DataFrame for verification
projected_nonrenewable_df.tail()


In [ ]:
projected_future_df=pd.merge(projected_df, projected_nonrenewable_df, on='year', how='left')
filt_projected_future_df=projected_future_df[projected_future_df['year'] == 2072]
filt_projected_future_df

In [ ]:
#Pie chart distribution for renewables vs non-renewables
# Extract the relevant columns for the pie chart
projected2072 = filt_projected_future_df[[renewables_consumption', 'non_renewables_consumption']]

# If the DataFrame is not empty, sum the values of each column
total_consumption = projected2072.sum()

# Pie chart configuration
explode = (0, 0)  # No slice exploded
colors = ["yellow", "lightcoral"]  # Colors for renewable and non-renewable slices

# Plot the pie chart
plt.figure(figsize=(8, 6))
plt.title("Renewable vs Non-Renewable Energy Consumption (Projected 2072)")

# Use the summed values for the pie chart
plt.pie(total_consumption, labels=total_consumption.index, 
        colors=colors, autopct="%1.1f%%", startangle=90)

# Ensure the pie chart is circular
plt.axis("equal")  
#Save the plot
plt.savefig('Resources/e_projected_energy.png')

# Show the plot
plt.show()

In [ ]:
six_continents = world_energy_df.loc[
    (
        (world_energy_df["country"] == "Asia") |
        (world_energy_df["country"] == "Africa") |
        (world_energy_df["country"] == "Australia") |
        (world_energy_df["country"] == "Europe") |
        (world_energy_df["country"] == "North America") |
        (world_energy_df["country"] == "South America")
    ) & (world_energy_df["year"] >= 1985)  
]
six_continents

In [ ]:
six_continents=six_continents.rename(columns={'country': 'continent'})
six_continents

In [ ]:
#VISUAL (SEZER)
# Plot energy consumption over time by continent
consumption_plot_1a = six_continents.hvplot.line(
    x='year', y='Total Energy Consumption', by='continent', 
    title="Total Energy Consumption Over Time by Continent",
    xlabel='Year', ylabel='Total Energy Consumption',
    line_width=2, height=400, width=1000
)
consumption_plot_1a

In [ ]:
#VISUAL
# Plotting the line plot by continent using Matplotlib
plt.figure(figsize=(8, 6))  # Set figure size

# Loop through each continent and plot its line
for continent in six_continents['continent'].unique():
    continent_data = six_continents[six_continents['continent'] == continent]
    plt.plot(continent_data['year'], continent_data['Total Energy Production'], label=continent, linewidth=2)

# Add title and labels
plt.title("Total Energy Production Over Time by Continent")
plt.xlabel('Year')
plt.ylabel('Total Energy Production')

# Show legend to identify the continents
plt.legend(title='Continent')

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
#Define dataframes to show renewable energy and non-renewable energy for 6 continents
renewable_energy_df=six_continents.loc[:,["continent", "year","renewables_consumption"]]

non_renewable_energy_df=six_continents.loc[:,["continent", "year","non_renewables_consumption" ]]


In [ ]:
#Pie chart showing renewable energy consumption in 2022 per continent
renewable_2022 = renewable_energy_df[renewable_energy_df['year'] == 2022]
non_renewable_2022 = non_renewable_energy_df[non_renewable_energy_df['year'] == 2022]

# Define color palettes for renewable and non-renewable pie charts
colors=["yellow","green","lightblue","orange","red","purple"]
explode=(0,0.1,0,0,0,0)
plt.figure(figsize=(8, 8))
plt.pie(
    renewable_2022['renewables_consumption'], 
    labels=renewable_2022['continent'], 
    autopct='%1.1f%%', 
    startangle=140, 
    colors=colors,
    explode=explode,
    shadow=True,
    
)
plt.axis("equal")
plt.title("Renewable Energy Consumption by Continent in 2022")
plt.show()

In [ ]:
# Filter data for the year 2022
year_2022_data = six_continents[six_continents['year'] == 2022]

In [ ]:
#VISUAL (Sezer)
# Renewables vs non-renewables energy consumption per continent
energy_mix_plot = year_2022_data.hvplot.bar(
    x='continent', y=['renewables_consumption', 'non_renewables_consumption'], 
    stacked=True, title="Energy Mix by Continent in 2022",
    xlabel='Continent', ylabel='Energy Source Contribution',
    height=400, width=800
)
energy_mix_plot

In [ ]:
# Q2 (Sezer) Is there a correlation between total energy consumption per capita and total energy 
#production per capita in different continents?

# Drop any rows with NaN values in the relevant columns

six_continents_cleaned = six_continents.dropna(subset=['total_prod_per_capita', 'total_cons_per_capita'])
# Perform the t-test between total production per capita and total consumption per capita
t_statistic, p_value = stats.ttest_ind(
    six_continents_cleaned['total_prod_per_capita'], 
    six_continents_cleaned['total_cons_per_capita'],
    equal_var=False
)

print(f"T-statistic: {t_statistic}, p-value: {p_value}")

In [ ]:
# Q2 (Sezer) #ANOVA TEST 
import warnings
warnings.filterwarnings('ignore')
# Extract individual groups decade
group0 = six_continents[six_continents["year"] == 2012]["total_cons_per_capita"]
group1 = six_continents[six_continents["year"] == 2013]["total_cons_per_capita"]
group2 = six_continents[six_continents["year"] == 2014]["total_cons_per_capita"]
group3 = six_continents[six_continents["year"] == 2015]["total_cons_per_capita"]
group4 = six_continents[six_continents["year"] == 2016]["total_cons_per_capita"]
group5 = six_continents[six_continents["year"] == 2017]["total_cons_per_capita"]
group6 = six_continents[six_continents["year"] == 2018]["total_cons_per_capita"]
group7 = six_continents[six_continents["year"] == 2019]["total_cons_per_capita"]
group8 = six_continents[six_continents["year"] == 2020]["total_cons_per_capita"]
group9 = six_continents[six_continents["year"] == 2021]["total_cons_per_capita"]
group10 = six_continents[six_continents["year"] == 2022]["total_cons_per_capita"]

stats.f_oneway(group0, group1, group2, group3, group4, group5, group6, group7, group8, group9,group10)